# Associated Hazard Extractor
Used to combine the association matrix with the main hazard definitions DF to slot associations into 'Upstream Hazards'.

## Step 1: Get Requirements

In [1]:
import pandas as pd

## Step 2: Open data/scores.xlsx. 
This contains the full 2D association matrix, 303 x 303; the first row and column are the names of the hazards. The cells contain the scores of the hazards. The scores are the result of prompting an LLM to generate the associations.

In [3]:
df = pd.read_excel('scores.xlsx')
df = df.set_index('Unnamed: 0')
df

,Downburst,Lightning (Electrical Storm),Thunderstorm,Coastal Flood,Estuarine (Coastal) Flood,Flash Flood,Fluvial (Riverine) Flood,Groundwater Flood,Ice-Jam Flood Including Debris,Ponding (Drainage) Flood,...,Road Traffic Accident,Explosive agents,International Armed Conflict (IAC),Non-International Armed Conflict (NIAC),Civil Unrest,Explosive Remnants of War,Environmental Degradation from Conflict,Violence,Stampede or Crushing (Human),Financial shock
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Downburst,NaN,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,...,3.0,3.0,2.0,3.0,3.0,3.0,4.0,2.0,4.0,3.0
Lightning (Electrical Storm),4.0,NaN,5.0,1.0,4.0,3.0,4.0,-1.0,4.0,4.0,...,2.0,4.0,1.0,2.0,2.0,3.0,-1.0,1.0,4.0,1.0
Thunderstorm,-1.0,4.0,NaN,3.0,4.0,4.0,4.0,4.0,3.0,1.0,...,1.0,3.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0
Coastal Flood,-1.0,3.0,3.0,NaN,4.0,3.0,3.0,4.0,4.0,4.0,...,2.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,4.0,4.0
Estuarine (Coastal) Flood,3.0,3.0,3.0,4.0,NaN,3.0,3.0,4.0,4.0,4.0,...,3.0,2.0,2.0,2.0,3.0,2.0,4.0,2.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Explosive Remnants of War,4.0,3.0,2.0,3.0,3.0,2.0,3.0,4.0,4.0,4.0,...,3.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0,4.0
Environmental Degradation from Conflict,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0
Violence,4.0,1.0,0.0,2.0,3.0,2.0,3.0,0.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0


## Step 3: Transform the 2D matrix into a 1D array of tuples, where each tuple contains the name of the hazard and the association score.

In [4]:
data = []
for index, row in df.iterrows():
    for column in df.columns:
        data.append((index, column, row[column]))

data[:5]

[('Downburst', 'Downburst', nan),
 ('Downburst', 'Lightning (Electrical Storm)', 4.0),
 ('Downburst', 'Thunderstorm', 4.0),
 ('Downburst', 'Coastal Flood', 4.0),
 ('Downburst', 'Estuarine (Coastal) Flood', 4.0)]

## Step 4: Create a DF from the 1D array.
We will drop any rows with a score of less than 4, as we correlate any score less than 4 to be a non association.

In [5]:
scores_df = pd.DataFrame(data, columns=['hazard1', 'hazard2', 'score'])
scores_df

,hazard1,hazard2,score
0,Downburst,Downburst,NaN
1,Downburst,Lightning (Electrical Storm),4.0
2,Downburst,Thunderstorm,4.0
3,Downburst,Coastal Flood,4.0
4,Downburst,Estuarine (Coastal) Flood,4.0
...,...,...,...
91199,Financial shock,Explosive Remnants of War,NaN
91200,Financial shock,Environmental Degradation from Conflict,NaN
91201,Financial shock,Violence,NaN
91202,Financial shock,Stampede or Crushing (Human),NaN


In [8]:
# Drop all rows with a score less than 4
scores_df = scores_df[scores_df['score'] >= 4]
scores_df

,hazard1,hazard2,score
82277,Fire,Mercury,5.0
20908,Earthquake,Ground Shaking (Earthquake),5.0
5402,Polluted Air,Pollution,5.0
82294,Fire,Methanol,5.0
61782,Measles (Human),Vaccine-Preventable Diseases (Human),5.0
...,...,...,...
49136,Antimicrobial Resistance,Malaria (Human),4.0
49142,Antimicrobial Resistance,Monkeypox (Human),4.0
49135,Antimicrobial Resistance,Dengue (Human),4.0
49130,Antimicrobial Resistance,Pertussis (Human),4.0


## Step 5: Open the main hazard_definitions spreadsheet.
We will then insert all the hazard associations of score larger or equal to 4 to the Upstream_Hazards column.

In [10]:
hazard_definitions = pd.read_excel("data/hazard_definitions.xlsx")
hazard_definitions

,Hazard_Code,Hazard_Category,Hazard_Subcategory,Hazard_Name,Hazard_Description,Upstream_Hazards,Excluded_Hazards,Synonyms,Keywords,Keywords_Operator,Questions
0,MH0001,METEOROLOGICAL AND HYDROLOGICAL,Convective-Related,Downburst,Downburst A downburst is a violent and damagin...,NaN,NaN,"Microburst, Macroburst, Wind Sear",Wind,OR,Did a downburst occur?
1,MH0002,METEOROLOGICAL AND HYDROLOGICAL,Convective-Related,Lightning (Electrical Storm),Lightning Lightning is the luminous manifestat...,NaN,NaN,"Bolt, Thunderbolt, Bolt-from-the-blue, Firebol...","Lightning, storm",OR,Was there lightning?
2,MH0003,METEOROLOGICAL AND HYDROLOGICAL,Convective-Related,Thunderstorm,Thunderstorm A thunderstorm is defined as one ...,MH0002,NaN,NaN,storm,OR,Was there a thunderstorm?
3,MH0004,METEOROLOGICAL AND HYDROLOGICAL,Flood,Coastal Flood,Coastal Flood Coastal flooding is most frequen...,NaN,NaN,"Storm Surge, Coastal inundation",Flood,OR,Did the even take place at the coast?
4,MH0005,METEOROLOGICAL AND HYDROLOGICAL,Flood,Estuarine (Coastal) Flood,Estuarine Flood Estuarine flooding is flooding...,MH0004,NaN,"Flood, Flooding, Coastal flooding",river,OR,Was the river flooding caused by coastal flood...
...,...,...,...,...,...,...,...,...,...,...,...
297,SO0004,SOCIETAL,Post-Conflict,Explosive Remnants of War,Explosive Remnants of War Explosive remnants o...,SO0003,NaN,"Unexploded ordnance, Abandoned explosive ordnance","explosive, crater, mines",AND,Were there any reports of explosive remnants o...
298,SO0005,SOCIETAL,Post-Conflict,Environmental Degradation from Conflict,Environmental Degradation from Conflict Enviro...,SO0003,NaN,"Ecological degradation, Environmental damage","pollution, war",AND,Were there any reports of environmental degrad...
299,SO0006,SOCIETAL,Behavioural,Violence,Violence Violence refers to the intentional or...,SO0003,NaN,NaN,violence,OR,Were there any reports of violence?
300,SO0007,SOCIETAL,Behavioural,Stampede or Crushing (Human),Stampede or Crushing Stampede or crushing is t...,SO0003,NaN,"Crush, Mass panic, Crowd disaster","stampede, crushing",OR,Were there any reports of stampede or crushing?


In [13]:
for pair in scores_df.itertuples():
    hazard1 = hazard_definitions[hazard_definitions["Hazard_Name"] == pair.hazard1]
    hazard2 = hazard_definitions[hazard_definitions["Hazard_Name"] == pair.hazard2]
    if len(hazard1) > 0 and len(hazard2) > 0:
        hazard1_id = hazard1["Hazard_Code"].values[0]
        hazard2_id = hazard2["Hazard_Code"].values[0]
        # Add the hazard2_id to the hazard1 row Upstream_Hazards column
        if hazard1["Upstream_Hazards"].values[0] == "None":
            hazard_definitions.loc[hazard_definitions["Hazard_Name"] == pair.hazard1, "Upstream_Hazards"] = hazard2_id
        else:
            hazard_definitions.loc[hazard_definitions["Hazard_Name"] == pair.hazard1, "Upstream_Hazards"] += ", " + hazard2_id

hazard_definitions

,Hazard_Code,Hazard_Category,Hazard_Subcategory,Hazard_Name,Hazard_Description,Upstream_Hazards,Excluded_Hazards,Synonyms,Keywords,Keywords_Operator,Questions
0,MH0001,METEOROLOGICAL AND HYDROLOGICAL,Convective-Related,Downburst,Downburst A downburst is a violent and damagin...,NaN,NaN,"Microburst, Macroburst, Wind Sear",Wind,OR,Did a downburst occur?
1,MH0002,METEOROLOGICAL AND HYDROLOGICAL,Convective-Related,Lightning (Electrical Storm),Lightning Lightning is the luminous manifestat...,NaN,NaN,"Bolt, Thunderbolt, Bolt-from-the-blue, Firebol...","Lightning, storm",OR,Was there lightning?
2,MH0003,METEOROLOGICAL AND HYDROLOGICAL,Convective-Related,Thunderstorm,Thunderstorm A thunderstorm is defined as one ...,"MH0002, MH0002, MH0005, MH0006, MH0007, MH0008...",NaN,NaN,storm,OR,Was there a thunderstorm?
3,MH0004,METEOROLOGICAL AND HYDROLOGICAL,Flood,Coastal Flood,Coastal Flood Coastal flooding is most frequen...,NaN,NaN,"Storm Surge, Coastal inundation",Flood,OR,Did the even take place at the coast?
4,MH0005,METEOROLOGICAL AND HYDROLOGICAL,Flood,Estuarine (Coastal) Flood,Estuarine Flood Estuarine flooding is flooding...,"MH0004, EN0015, EN0016, BI0007, EN0010, MH0012...",NaN,"Flood, Flooding, Coastal flooding",river,OR,Was the river flooding caused by coastal flood...
...,...,...,...,...,...,...,...,...,...,...,...
297,SO0004,SOCIETAL,Post-Conflict,Explosive Remnants of War,Explosive Remnants of War Explosive remnants o...,"SO0003, MH0008, MH0023, MH0020, MH0019, MH0018...",NaN,"Unexploded ordnance, Abandoned explosive ordnance","explosive, crater, mines",AND,Were there any reports of explosive remnants o...
298,SO0005,SOCIETAL,Post-Conflict,Environmental Degradation from Conflict,Environmental Degradation from Conflict Enviro...,"SO0003, BI0027, BI0028, BI0030, TL0031, TL0028...",NaN,"Ecological degradation, Environmental damage","pollution, war",AND,Were there any reports of environmental degrad...
299,SO0006,SOCIETAL,Behavioural,Violence,Violence Violence refers to the intentional or...,"SO0003, TL0038, SO0008, SO0007, SO0005, SO0004...",NaN,NaN,violence,OR,Were there any reports of violence?
300,SO0007,SOCIETAL,Behavioural,Stampede or Crushing (Human),Stampede or Crushing Stampede or crushing is t...,"SO0003, BI0007, CH0024, CH0020, CH0013, CH0008...",NaN,"Crush, Mass panic, Crowd disaster","stampede, crushing",OR,Were there any reports of stampede or crushing?


## Step 6: Update the hazard_definitions spreadsheet and save it to data folder.

In [15]:
hazard_definitions.to_excel("data/hazard_definitions_updated.xlsx", index=False)